# 磁盘管理
- df：查看各个盘的使用情况
- du：查看具体使用情况
     - du -sm /*:查看根目录下一些大目录的使用情况
- mac或者linux挂载我们的一些本地磁盘（比如U盘）或者文件到磁盘中
    - mount /dev/upan /mnt/upan-use
    - umount:取消挂载，-f强制取消

# 磁盘
- 磁盘分区：将硬盘划分成多个逻辑存储单元
- 逻辑存储单元：分区
- 各种存储盘（比如U盘，固态硬盘）都必须经过分区、格式化，成为文件系统后才能被计算机作为文件存储设备使用（而文件系统需要挂载到某个目录上才能够对其中的文件进行读写）

## 分区的益处
- 1.限制应用或用户的可用空间
- 2.允许同一块硬盘安装不同的操作系统
- 3.可用给虚拟内存一个单独的分区
- 4.提高硬盘的使用费性能

- 磁盘由磁道，磁道由扇区（512字节构成）构成

## 磁盘如何分区
- MBR:Main Boot Record(主引导记录):告诉硬盘如何分区，写在磁盘的第一个扇区中
- 特点：
    - 1.支持四个主分区
    - 2.使用扩展分区和逻辑分区，可以支持15个分区
    - 3.允许最大分区和磁盘大小为2T
    
>常用的分区工具有fdisk，sfdisk,cfdisk，分区完成后，还需要对各个分区进行格式化



### fdisk来进行分区
- df命令：查看硬盘使用情况，参数-h,或者-hT
- IDE接口硬盘：/dev/dh[a-z], SCSI接口硬盘：/dev/sd[a-z]
- 添加过程
    - 首先使用virtualbox给虚拟机添加虚拟硬盘
    - 使用命令ls /dev/sd查看所有的硬盘，看不了就直接ls /dev
    - 分区：使用命令fdisk /dev/sda，然后根据相应的提示创建分区，相关命令如下：
        - n：新建>p：创建主分区>分区编号>起始扇区>设置分区大小>p(查看)>w(保存),d是删除分区
    - 内核重新读取新的分区表
        - partprobe /dev/sdb
    - 创建文件系统，Linux支持很多文件系统，ext4和xfs,windows下文件系统NTFS FAT32
        - 命令：mkfs.ext4 /dev/sdb1，注意一定是格式化分区而不是一整块硬盘
    - 挂载：将文件系统连接到目录结构中——即将目录和一个硬盘进行挂钩
        - 手动挂载 ：重启后会失效，用于检测已经格式化的设备，是否可以和预期一样的进行工作，使用：mount 文件系统 挂载点（目录）
        - 永久 挂载：就要将设备添加到/etc/fstab中去了
            - vim /etc/fstab
            - 挂载写入的文件格式如下：
                - 该文件由六个部分组成
                    - 第一部分：要使用的设备——可以使用UUID,也可以使用设备文件 blkid：查看设备的UUID号， blkid主要用来对系统的块设备（包括 交换分区）所使用的文件系统类型、LABEL、UUID等信息进行查询，要使用这个命令必须安装e2fsprogs软件包
                    - 第二部分：写挂载点，已经存在直接写，不存在使用mkdir来创建
                    - 第三部分：文件系统类型
                    - 第四部分：挂载时，应用于设备的自定义选项，defaults是必须的
                    - 第五和第六部分：转存标志和fsck顺序
                        - 转存标志：用于生成设备内容的备份
                            - 0：表示不需要备份
                            - 1：表示需要备份
                        - fsck：当有多个文件系统需要检查时，检查的顺序 
                            -  0：表示不需要检查，开机直接挂载
                            - 1：表示检查后第一个挂载
                            - 2：表示检查后第二个挂载
                    - 示例：/dev/sdb1 挂载点（即目录） ext4 defaults 0 0
                - 使用mount -a命令，检查书写格式是否有误，一定要检查，否则有错重新开机将进入警告模式